In [ ]:
import re
import PyPDF2
import pandas as pd

def return_pdf_text(pdf_name):
    pdfFileObj = open(pdf_name, 'rb')

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    abs_text = ''
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        testPage = pageObj.extract_text()

        if re.search('JVIR.*?S\d{1,4}$', testPage): # cleaning the text during runtime
            if re.search('JVIR.*?S\d{1,4}$', testPage, flags=re.S):
                if len(re.search('JVIR.*?S\d{1,4}$', testPage, flags=re.S).group()) > 50:
                    pass
                else:
                    testPage = re.sub('JVIR.*?S\d{1,4}$', '', testPage, flags=re.S)

        else:
            if re.search('S\d{1,4}.*$', testPage, flags=re.S):
                if len(re.search('S\d{1,4}.*$', testPage, flags=re.S).group()) > 50:
                    pass
                else:
                    testPage = re.sub('S\d{1,4}.*$', '', testPage, flags=re.S)
            
        abs_text += testPage + '\n'
    
    pdfFileObj.close()
    return abs_text

In [ ]:
pdf_text = return_pdf_text('ev_pdf.pdf')

In [ ]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [ ]:
c = 0
for i in re.split('Abstract No.', pdf_text, flags=re.S)[1:]:
    
    title = ''
    auth_aff = ''
    abs_text = ''

    title_flag = True
    auth_flag = False
    text_flag = False


    for itxt in i.split('\n')[1:]:

        if re.search('ABSTRACT OF THE YEAR', itxt):
            continue

        if re.search('.*?\d,|.*?\d;', itxt) and title_flag:
            title_flag = False
            auth_flag = True

        if title_flag:
            title += itxt + ' '

        if auth_flag:
            if re.search('Purpose:', itxt):

                abs_text += re.search('Purpose:.*', itxt).group()
                itxt = re.sub('Purpose:.*', '', itxt)
                auth_flag = False
                text_flag = True
            auth_aff += itxt + ' '

        if text_flag:

            abs_text += itxt + ' '

    if re.search('Disclaimer:.*?:', abs_text):

        subTxt = re.search('Disclaimer:.*?:', abs_text).group()
        yahase = re.findall('[A-Za-z]*?:', subTxt)[1]

        bef = re.sub('Disclaimer:.*?:', '', abs_text, flags=re.S)
        aft = re.search(f'{yahase}.*', abs_text, flags=re.S).group()
        abs_text = bef + ' ' + aft

    else:
        pass

    title = re.sub('▪', '', title)
    author = auth_aff.split(';')[0]
    affiliation = '; '.join(auth_aff.split(';')[1:])

    print(f'tit :- {title}')
    print(f'auth :- {author}')
    print(f'aff :- {affiliation}')
    
    print('======'*18, c+1)
    
    dic['session_id'].append(f'S{c+1}')
    dic['news_type'].append('Session')
    dic['source_id'].append('')
    dic['manual_id'].append('')
    dic['article_title'].append(title)
    dic['url'].append(f'https://www.jvir.org/pb-assets/Health%20Advance/journals/jvir/JVIR_34_3_S_Online-1677513212153.pdf')
    dic['authors'].append(author)
    dic['author_affiliation'].append(affiliation)
    dic['abstract_text'].append(abs_text)
    dic['date'].append('')
    dic['start_time'].append('')
    dic['end_time'].append('')
    dic['location'].append('')
    dic['session_title'].append(title)
    dic['session_type'].append('')
    dic['category'].append('')
    dic['sub_category'].append('')
    dic['disclosure'].append('')
    
    c += 1

In [ ]:
df = pd.DataFrame(dic)
df

In [ ]:
df.to_excel('SIR_pdf_add.xlsx', index=False)